# Orthogonalization & Peel Off

In this notebook, I add functions for peel-off process and a more robust orthogonalization procedure. These are standalone functions here and must be worked into the iterative algorithm. Below I outline what needs to be done to include them in the algorithm.

__TO-DO__:
- Modify **separation** and wrapper **decomposition** functions to include optional argument for orthogonalization function of choice. Default both to Gram-Schmidt. So decomposition() will take it in as an argument and pass it to separation() which in turn will pass it to orthoganalization().
- Modify **refinement** and wrapper **decomposition** functions to include boolean argument for peel off, and give both a default, i.e. peel_off=False. So decomposition() will take it in as an argument and pass it to refinement(). Add code in refinement along the lines of "if peel_off: z = peel_off(z,s)" inside the refining loop.

In [1]:
import numpy as np

## Peel Off

In [2]:
def peel_off(z, s):
    """
    Removed a source vector from the whitened matrix.
    
    Parameters
    ----------
    z: numpy.ndarray
        Matrix used in separation and refining steps of decomposition.
    s: numpy.ndarray
        Source vector to be removed from the whitened matrix.
    
    Returns
    -------
        numpy.ndarray
            Matrix with source array removed.
            
    Examples
    --------
    >>> z = np.array([[1, 4, 6, 9],
              [3, 14, 62, 12],
              [7, 43, 16, 8]])
    >>> s = [2, 2, 1, 3]
    >>> peel_off(z, s)
    array([[-1,  2,  5,  6],
           [ 1, 12, 61,  9],
           [ 5, 41, 15,  5]])
    """
    return z - s

In [3]:
z = np.array([[1, 4, 6, 9],
              [3, 14, 62, 12],
              [7, 43, 16, 8]])
print(z)
z.shape

[[ 1  4  6  9]
 [ 3 14 62 12]
 [ 7 43 16  8]]


(3, 4)

## Deflate Function

In [4]:
def deflate(w, B):
    """
    Step 2b from Negro et al. (2016): wi(n) = wi(n) - BB^{T} * w_i(n)
    Note: this is not true orthogonalization, such as the Gram–Schmidt process.
    This is dubbed in paper "source deflation procedure."

    Parameters
    ----------
        w: numpy.ndarray
            Vectors for which we seek orthogonal matrix.
        B: numpy.ndarray
            Matrix to 'deflate' w by.

    Returns
    -------
        numpy.ndarray
            'Deflated' array.

    Examples
    --------
        >>> w = np.array([[5, 6], [23, 29]])
        >>> B = np.array([[3, 3], [3, 3]])
        >>> orthogonalize(w, B)
        array([[-499, -624],
               [-481, -601]])
    """
    w = w - np.dot(B, np.dot(B.T, w))
    return w

## Gram-Schmidt orthogonalization

In [5]:
def gram_schmidt(w, B):
    """
    Gram-Schmidt orthogonalization.

    Parameters
    ----------
        w: numpy.ndarray
            Vector we are orthogonalizing against columns of B.
        B: numpy.ndarray
            Matrix to 'deflate' w by. Should contain float dtype.

    Returns
    -------
        numpy.ndarray
            'Deflated' array.

    Examples
    --------
        >>> w = np.array([7, 4, 6])
        >>> B = np.array([[ 1. ,  1.2,  0. ],
                          [ 2. , -0.6,  0. ],
                          [ 0. ,  0. ,  0. ]])
        >>> orthogonalize(w, B)
        array([-2.66453526e-15,  0.00000000e+00,  6.00000000e+00])
    """
    projw_a = 0
    for i in range(B.shape[1]):
        a = B[:, i]
        if np.all(a == 0):
            continue
        projw_a = projw_a + (np.dot(w, a) / np.dot(a, a)) * a
    w = w - projw_a
    return w

## "Parent" Orthogonalize Wrapper

In [6]:
def orthogonalize(w, B, fun=gram_schmidt):
    """
    Step 2b from Negro et al.
    Performs orthogonalization using selected process.
    
     Parameters
    ----------
        w: numpy.ndarray
            Vector we are orthogonalizing against columns of B.
        B: numpy.ndarray
            Matrix to orthogonize w against. Should contain float dtype.
        fun: function object name
            What function to use for orthogonalizing process
            Current options are
                - gram_schmidt (default)
                - deflate

    Returns
    -------
        numpy.ndarray
            Orthogonilized array.

    Examples
    --------
        >>> w = np.array([[5, 6], [23, 29]])
        >>> B = np.array([[3, 3], [3, 3]])
        >>> fun = deflate
        >>> orthogonalize(w, B, fun)
        array([[-499, -624],
               [-481, -601]])
    """
    return fun(w,B)

In [7]:
w = np.array([[5, 6], [23, 29]])
B = np.array([[3, 3], [3, 3]])

In [8]:
fun = deflate
orthogonalize(w, B, fun)

array([[-499, -624],
       [-481, -601]])

In [9]:
fun = deflate
orthogonalize(w, B) # default = Gram-Schmidt

array([[ -6., -46.],
       [ 12., -23.]])